# Priprema podataka

**Ucitavanje podataka iz datoteke**
  - u prvoj liniji su reci za koje treba naci regex (skup M)
  - u drugoj liniji su reci za koje ne treba naci regex (skup U)
  - reci u razdvojene sa: ", "

In [1]:
def readFile(filename):
    with open(filename, 'r') as f:
        match = [word for word in f.readline().split(", ")]
        # uklanjanje novog reda iz poslednje reci
        match[-1] = match[-1][:-1]
        unmatch = [word for word in f.readline().split(", ")]
        
    return match, unmatch

In [4]:
# M i U skupovi reci
match, unmatch = readFile("/content/example_1.txt")
print("M skup: ", match)
print("U skup: ", unmatch)

M skup:  ['afoot', 'catfoot', 'dogfoot', 'fanfoot', 'foody', 'foolery', 'foolish', 'fooster', 'footage', 'foothot', 'footle', 'footpad', 'footway', 'hotfoot', 'jawfoot', 'mafoo', 'nonfood', 'padfoot', 'prefool', 'sfoot', 'unfool']
U skup:  ['Atlas', 'Aymoro', 'Iberic', 'Mahran', 'Ormazd', 'Silipan', 'altared', 'chandoo', 'crenel', 'crooked', 'fardo', 'folksy', 'forest', 'hebamic', 'idgah', 'manlike', 'marly', 'palazzi', 'sixfold', 'tarrock', 'unfold']


In [5]:
# Broj reci u skupovima
num_m = len(match)
num_u = len(unmatch)
print(num_m)
print(num_u)

21
21


**Karakteri koji se pojavljuju u skupu M**

In [6]:
def charsInSet(wordSet):
    chars = []
 
    for word in wordSet:
        for c in word:
            if c not in chars:
                chars.append(c)
 
    chars.sort()
 
    return chars

In [7]:
chars_in_M = charsInSet(match)
print(chars_in_M)

['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'y']


**Opsezi reci (partial ranges) koji se javljaju u skupu M**

In [8]:
def makeRanges(chars_in_M):
    ranges = []

    i = 0
    #za svako slovo niza proveravamo
    while i < len(chars_in_M)-1:
        distance = 0
        for j in range(i+1, len(chars_in_M)):
            if ord(chars_in_M[j]) - ord(chars_in_M[i]) == distance + 1:
                distance += 1
            else:
                if chars_in_M[i] != chars_in_M[j-1]:
                    ranges.append(chars_in_M[i] + '-' + chars_in_M[j-1])
                i = j
                break

    return ranges

In [9]:
ranges = makeRanges(chars_in_M)
print(ranges)

['c-j', 'l-p', 'r-u']


**n-grami**

In [10]:
def ngram(M, U):
    res = {}

    # n-grami su duzine od 2 do 4
    for n in range(2, 5):
        # prolazimo kroz sve reci iz M i iz U 
        # (radi ako su skupovi iste duzine)
        for i in range(0, len(M)):
            word_m = M[i]
            word_u = U[i]
            
            word_m_visited = False
            word_u_visited = False
            
            # pravimo n-grame od jedne reci iz M i jedne iz U
            ngrams_m = zip(*[word_m[i:] for i in range(n)])
            ngrams_u = zip(*[word_u[i:] for i in range(n)])

            gram_m = ["".join(gr) for gr in ngrams_m]
            gram_u = ["".join(gr) for gr in ngrams_u]

            # ne smemo imati ponavljanje n-grama u istoj reci (pravimo skup)
            gram_m = set(gram_m)
            gram_u = set(gram_u)

            # azuriramo score za ngram u zavisnosti u kom skupu se nalazi
            for g in gram_m: # povecavamo score ako je u M
                if g not in res:
                    res[g] = 1
                elif g in res: 
                    res[g] += 1

            for g in gram_u: # smanjujemo score ako je u U
                if g not in res:
                    res[g] = -1
                elif g in res:
                    res[g] -= 1

    return res

In [11]:
ngrams = ngram(match, unmatch)
ngrams = sorted(ngrams.items(), key=lambda x: x[1], reverse=True)

# ngram_subset je najmanji podskup od ngrams tako da je skor reci bar |M|
ngram_subset = []
score = 0

for i in range(len(ngrams)):
    if ngrams[i][1] > 0: # azuriramo samo ako je skor pozitivan
        score += ngrams[i][1]
        ngram_subset.append(ngrams[i][0])

        if score >= num_m:
            break 

print(ngram_subset)

['foo']


**Terminal i Function skupovi**

In [12]:
# . je placeholder za dete cvor
FUNCTION_SET = [".*", ".+", ".?", ".{.,.}+", # possessive quantifiers
                "(.)",                          # group
                "[.]",                          # character class
                "[^.]",                         # negated character
                "..",                           # concatenator (binary node) mislim da treba da promenimo prikaz ovog noda tipa ` jer je kod njih tacka na sredini a ovo znaci concat
                ".|.",                          # disjunction
                ]

In [13]:
TERMINAL_SET = ["a-z", "A-Z", "0-9", "^", "$", "%", # instance independent terminals
                "\w", "\W", "\d", "\D", "\b", "\B", "\A", "\Z", "\s", "\S"
               ]

In [14]:
# dodajemo sve karaktere iz M u terminal set
TERMINAL_SET.extend(chars_in_M)

# upisujemo n_grame u terminal set
TERMINAL_SET.extend(ngram_subset)

# ispisujemo range-ove u terminal set
TERMINAL_SET.extend(ranges)

print(TERMINAL_SET)

['a-z', 'A-Z', '0-9', '^', '$', '%', '\\w', '\\W', '\\d', '\\D', '\x08', '\\B', '\\A', '\\Z', '\\s', '\\S', 'a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'y', 'foo', 'c-j', 'l-p', 'r-u']


# Formiranje jedinke

In [15]:
import re
import random

Funkcija *getRandom()* bira jedan element ili iz skupa Terminal ili iz skupa Function

In [16]:
def getRandom():
      pickSet = random.choice(['f', 't'])
      if pickSet == 't':
        value = random.choice(FUNCTION_SET)
        if value in [".{.,.}+"]:
          return value, 3
        elif value in [".|.", ".."]:
          return value, 2
        else:
          return value, 1
      else:
        value = random.choice(TERMINAL_SET)
        return value, 0

Klasa *Node* predstavlja apstraktno drvo na osnovu koga se dobija regularni izraz

In [17]:
class Node(object):
  def __init__(self, depth, root):
    self.depth = depth
    self.value = ""
    self.childrenNum = 0
    self.id = -1
    
    if root:
      self.value = "."
      self.childrenNum = 2
    else:
      self.value, self.childrenNum = getRandom()
    
    self.left = None
    self.right = None
    self.third = None

    if self.childrenNum == 3:
      self.left= Node(depth+1, False)
      self.right = Node(depth +1, False)
      self.third = Node(depth+1, False)
    elif self.childrenNum == 2:
      self.left = Node(depth+1, False)
      self.right = Node(depth+1, False)
    elif self.childrenNum == 1:
      self.left = Node(depth+1, False)  

Obilazimo drvo BFS-om i oznacavamo sve cvorove kao neposecene

In [19]:
def unvisit(n):

  q = []
  q.append(n)

  while len(q) >= 1:
    top = q.pop(0)

    if top.id > -1:
      top.id = -1
      i = top.childrenNum
      if i == 1:
        q.append(top.left)
        
      elif i == 2:
        q.append(top.left)
        q.append(top.right)
        
      elif i == 3:
        q.append(top.left)
        q.append(top.right)
        q.append(top.third)

Obilazimo drvo BFS-om i numerisemo redom cvorove

In [20]:
def treeNumeration(n):

  unvisit(n)

  parentMap = {}
  q = []
  q.append(n)
  num = 0

  while len(q) >= 1:
    top = q.pop(0)

    if top.id == -1:
      top.id = num
      num += 1
      i = top.childrenNum
      if i == 0:
        parentMap[num-1] = [-1]
      elif i == 1:
        q.append(top.left)
        parentMap[num-1] = [top.left]
      elif i == 2:
        q.append(top.left)
        q.append(top.right)
        parentMap[num-1] = [top.left, top.right]
      else:
        q.append(top.left)
        q.append(top.right)
        q.append(top.third)
        parentMap[num-1] = [top.left, top.right, top.third]
  
  # u mapi se uvek prvo navodi indeks levog, 
  # zatim desnog (ako postoji) i na kraju 
  # treceg deteta(ako postoji)
  return parentMap

Formiramo string koji predstavlja regularni izraz na osnovu stabla

In [21]:
def treeToString(node):

  if node.value in TERMINAL_SET:
    if node.value == "%":
      return "."
    return node.value
  
  rl= treeToString(node.left)
  if node.childrenNum == 2:
    rr = treeToString(node.right)
  if node.childrenNum == 3:
    rr = treeToString(node.right)
    rt = treeToString(node.third)
  
  if node.value in FUNCTION_SET:
    if node.value == ".*":
      string = rl + "*"
      return string
    if node.value == ".+":
      string = rl + "+"
      return string
    if node.value == ".?":
      string = rl + "?"
      return string
    if node.value == "(.)":
      string = "(" + rl + ")"
      return string
    if node.value == "[.]":
      string = "[" + rl + "]"
      return string
    if node.value == "[^.]":
      string = "[^" + rl + "]"
      return string
    if node.value == "..":
      string = rl + rr
      return string
    if node.value == ".|.":
      string = rl + "|" + rr
      return string
    if node.value == ".{.,.}+":
      string = rl + "{" + rr + "," + rt + "}+"
      return string

    # print("MATCHOVO SE SA FUNCTIONAL A NIJE FUNCTIONAL " + node.value)
  
  #ako je dosao do ovde je root
  string = rl + rr

  return string


Klasa *Individual* predstavlja jedinku koju koristimo za generisanje populacije

Svaka jedinka ima 2 vrste fitnesa:
- funkcija n_m - n_u treba da se maksimizuje
- duzina r treba da se minimizuje

r je trenutni regex, n_m je broj reci iz M koje su poklopljene sa r, n_u je broj reci iz U koje su poklopljene sa r

In [82]:
class Individual:
    def __init__(self, setM, setU):
        # code je apstrakto drvo koje cuva odredjeni regex u sebi
        self.code = self.initialize()
        self.wi = 10
        # setM i setU su ulazni skupovi M i U
        self.setM = setM.copy()
        self.setU = setU.copy()

        #n_m - n_u - maximize
        self.fitnessFunction = self.calculateFitnessFunction()
        # length(r) - minimize
        self.fitnessRegex = self.calculateFitnessRegex()
        # fitness racunamo kao wi*(n_m - n_u) - length(r)
        self.fitness = self.finalFitness()

    # inicijalizujemo code jedinke
    def initialize(self):
      generated = False
      while not generated:
        n = Node(0, True)
        treeString = treeToString(n)
        try:
          re.compile(treeString)
          # ako kompilacija regexa ne izazove exception,
          # onda je to validan regex i prihvatamo ovu jedinku
          generated = True 
        except Exception:
          generated = False
        
      return n

    # provera da li je dobijeni regex validan
    def isFeasible(self):
      treeString = treeToString(self.code)
      try:
        re.compile(treeString)
        return True
      except Exception:
        return False

    def __lt__(self, other):
      #zelimo da maksimizujemo fitnes
      return self.fitness > other.fitness
    
    def __str__(self):
      treeString = treeToString(self.code)
      return treeString

    def calculateFitnessFunction(self):
        n_m = 0
        n_u = 0
        regex = treeToString(self.code)

        for wordM, wordU in zip(self.setM, self.setU): 
          # vratice nam listu stringova koji se poklapaju
          matchM = re.findall(regex, wordM)
          matchU = re.findall(regex, wordU)

          foundM = False
          foundU = False

          for m in matchM:
            # u slucaju | - m moze imati vise elemenata u sebi
            for elem in m:
                if elem != "":
                    if len(elem) == len(wordM) or elem in wordM:
                        n_m += 1
                        foundM = True
                        break
            if foundM:
                break
            
          for m in  matchU:
            # u slucaju | - m moze imati vise elemenata u sebi
            for elem in m:
                if elem != "":
                    if len(elem) == len(wordU) or elem in wordU:
                        n_u += 1
                        foundU = True
                        break
            if foundU:
                break

        return n_m - n_u

    def calculateFitnessRegex(self):
        regex = treeToString(self.code)
        return len(regex)

    def finalFitness(self):
      try:
        return self.wi * self.calculateFitnessFunction() - self.calculateFitnessRegex()
      except Exception:
        return -100000

# Parametri genetskog programiranja

In [83]:
# pocetni parametri (zasnovani na dokumentaciji)
POPULATION_SIZE = 500
GENERATIONS_NUM = 1000
POPULATION_NUM = 32
TOURNAMENT_SIZE = 7
MUTATION_PROB = 0.1
ELITIZM_SIZE = int(0.02 * POPULATION_SIZE)

# Genetesko programiranje

Selekcija je klasicna, od sedam slucajno odabranih jedinki biramo onu sa najboljim fitnesom i njen indeks vracamo

In [84]:
import copy

In [85]:
def selection(population):
  betsFitness = float('-inf')
  bestIndex = -1

  for i in range(TOURNAMENT_SIZE):
    index = random.randrange(len(population))
    if population[index].fitness > betsFitness:
      betsFitness = population[index].fitness
      bestIndex = index
      
  return bestIndex

Pokusavamo da nadjemo cvor koji je numerisan brojem position, i njegovo dete (levo-0, desno-1, trece-2), menjamo adresom podstabla address

In [77]:
def replace(root, position, child, address):
  
  red = []
  red.append(root)
  found = False

  while not found:
    node = red.pop(0)
    if node.id == position:
      found = True
      if child == 0:
        node.left = address
      elif child == 1:
        node.right = address
      else:
        node.third = address
    else:
      children = node.childrenNum
      if children == 1:
        red.append(node.left)
      elif children == 2:
        red.append(node.left)
        red.append(node.right)
      elif children == 3:
        red.append(node.left)
        red.append(node.right)
        red.append(node.third)

Jednopoziciono ukrstanje

In [78]:
def crossover(parent1, parent2, child1, child2):
  
  map1 = treeNumeration(parent1.code)
  map2 = treeNumeration(parent2.code)
  
  parent1Size = len(map1)
  parent2Size = len(map2) 

  breakpoint = -1
  if parent1Size <= parent2Size:
    breakpoint = random.randrange(parent1Size)
  else:
    breakpoint = random.randrange(parent2Size)

  find = breakpoint
  
  if find == 0:
    # root je izabran
    child1.code = copy.deepcopy(parent2.code)
    child2.code = copy.deepcopy(parent1.code)
  else:
    child1.code = copy.deepcopy(parent1.code)
    child2.code = copy.deepcopy(parent2.code)
    
    unvisit(child1.code)
    unvisit(child2.code)

    #znamo da ce cvorovi biti isto numerisani
    mapChild1 = treeNumeration(child1.code)
    mapChild2 = treeNumeration(child2.code)
    
    map1Keys = mapChild1.keys()
    map2Keys = mapChild2.keys()

    replaceAtPositionParent1 = -1
    childAdress1 = -1
    side1 = -1
    replaceAtPositionParent2 = -1
    childAdress2 = -1
    side2 = -1

    for i in map1Keys:
      children = mapChild1[i]
      index = 0
      for child in children:
        if child == -1:
          continue
        if find == child.id:
          # id roditelja podstabla koje menjamo
          replaceAtPositionParent1 = i
          # pokazivac na podstablo koje menjamo
          childAdress1 = child
          # broj koji sugerise da li je levo(0), desno(1), ili trece(2) dete
          side1 = index
        else:
          index += 1

    # isto i ovde
    for i in map2Keys:
      children = mapChild2[i]
      index = 0
      for child in children:
        if child == -1:
          continue
        if find == child.id:
          replaceAtPositionParent2 = i
          childAdress2 = child
          side2 = index
        else:
          index += 1
  
    replace(child1.code, replaceAtPositionParent1, side1, childAdress2)
    replace(child2.code, replaceAtPositionParent2, side2, childAdress1) 

    if not child1.isFeasible():
      child1.code = copy.deepcopy(parent1.code)
    if not child2.isFeasible():
      child2.code = copy.deepcopy(parent2.code)


Mutaciju radimo sa verovatnocom 0.1

In [79]:
def mutation(individual):
  q = random.random()
  
  if MUTATION_PROB > q:
    # zelimo da pre mutacije sacuvamo trenutno drvo odnosno kod 
    oldCode = copy.deepcopy(individual.code)
    mapaSuseda = treeNumeration(individual.code)
    choiceRange = len(mapaSuseda)

    index = random.randrange(choiceRange)
    
    # obilazimo drvo dok se ne pozicioniramo na cvor sa datim indeksom
    previousValue = ""
    found = False

    q = []
    q.append(individual.code)

    while not found:
      n = q.pop(0)
      if n.id == index:
        # dosli smo u cvor sa datim indeksom
        found = True
        previousValue = n.value
        if n.value in FUNCTION_SET:
          # znamo da je onda u pitanju neki unutrasnji cvor
          newValue = random.choice(FUNCTION_SET)
          n.value = newValue
          children = n.childrenNum

          if n.value in [".*", ".+", ".?", "(.)", "[.]", "[^.]"] and children != 1:
            n.right = None
            if children == 3:
              n.third = None
            n.childrenNum = 1
          elif n.value in ["..", ".|."] and children != 2:
            if children == 1:
              n.right = Node(n.depth+1, False)
            else:
              # znaci da ima troje dece
              n.third = None
            n.childrenNum = 2
          elif n.value == ".{.,.}+" and children != 3:
            if children == 1:
              n.right = Node(n.depth+1, False)
              n.third = Node(n.depth+1, False)
            else:
              # znaci da ima 2 dece
              n.third = Node(n.depth, False)
            n.childrenNum = 3
          
          if not individual.isFeasible():
            n.value = previousValue
            individual.code = oldCode
        else:
          # znamo da je onda u pitanju neki list i nivu vrednost bitramo iz 
          # Terminal skupa
          newValue = random.choice(TERMINAL_SET)
          n.value = newValue
          if not individual.isFeasible():
            n.value = previousValue
      else:
        children = n.childrenNum
        if children == 0:
          continue
        elif children == 1:
          q.append(n.left)
        elif children == 2:
          q.append(n.left)
          q.append(n.right)
        elif children == 3:
          q.append(n.left)
          q.append(n.right)
          q.append(n.third)

In [80]:
def genetic_programming():
    population = [Individual(match, unmatch) for _ in range(POPULATION_SIZE)]
    newPopulation = [Individual(match, unmatch) for _ in range(POPULATION_SIZE)]

    solutions = []
    print(ELITIZM_SIZE)

    for i in range(GENERATIONS_NUM):
        population.sort()
        newPopulation[:ELITIZM_SIZE] = population[:ELITIZM_SIZE]

        # ako smo nasli jedinku koja zadovoljava 2 od 3 uslova
        # cuvamo je u solutions
        if population[0].fitnessFunction == num_m:
            solutions.append(population[0])

        for j in range(ELITIZM_SIZE, POPULATION_SIZE, 2):
            parent1Index = selection(population)
            parent2Index = selection(population)

            crossover(population[parent1Index], population[parent2Index], newPopulation[j], newPopulation[j+1])

            mutation(newPopulation[j])
            mutation(newPopulation[j+1])

            newPopulation[j].fitness = newPopulation[j].finalFitness()
            newPopulation[j+1].fitness = newPopulation[j+1].finalFitness()

        population = newPopulation

    # ako nismo imali "savrsenu" jedinku,
    # uzimamo najbolju jedinku populacije
    if len(solutions) <= 0:
        population.sort()
        solutions.append(population[0])

    return solutions

In [81]:
res = []
POPULATION_NUM = 10
for i in range(POPULATION_NUM):
    res.append(genetic_programming())

res.sort()
print("Best solution: ", res[0][0])

10
10
10
10
10
10
10
10
10
10
Best solution:  foo
